In [ ]:
%matplotlib inline
from lightkurve import search_targetpixelfile
from lightkurve import TessTargetPixelFile
import lightkurve as lk
import numpy as np

In [ ]:
# Download the pixelfile for a given star
# A quarter means a quarter of the year
pixelFile = search_targetpixelfile('KIC 6922244', author="kepler", cadence="long", quarter=4).download()

# Show a single snapshot
pixelFile.plot(frame=42)

In [ ]:
# We'll combine the individual frames into a lightkurve
# Aperture masks make the image look better for analysis
lc = pixelFile.to_lightcurve(aperture_mask=pixelFile.pipeline_mask)
lc.plot()

# We may find it easier to spot the pattern if we flatten the curve
flat_lc = lc.flatten()
flat_lc.plot()

# Phase-fold the light curve to verify that the period and transmit time correspond to the transit signal
# This puts the frequency spikes on top of each other if we get the period right
folded_lc = flat_lc.fold(period=3.5225)
folded_lc.plot()

# How to discover the correct peroid?
# Use a periodogram to show all the repetitive patterns in your graph
# Gives you the most likely candidate

# Periodograms are an estimate of the spectral density of a signal
# (An estimation of what the Fourier Trasform of the data would look like if it were a countinous function https://arxiv.org/pdf/1703.09824.pdf)

period = np.linspace(1, 5, 10000)
# BLS = Box Least Squares
bls = lc.to_periodogram(method="bls", period=period, frequency_factor=500)
bls.plot()

# Period value corresponding to the highest peak in the periodogram
planet_x_period = bls.period_at_max_power

planet_x_t0 = bls.transit_time_at_max_power
planet_x_dur = bls.duration_at_max_power

# Folding can yield a lot of information about the planet
# The depth can tell you about the size, etc.
ax = lc.fold(period=planet_x_period, epoch_time=planet_x_t0).scatter()
ax.set_xlim(-2, 2)

print(planet_x_period)
print(planet_x_t0)
print(planet_x_dur)

In [ ]:
# Use TESS data. That's being updated regularly. Kepler is already well analysed.

# Downloading MAST data
# Visit the MAST portal:
# https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html

# Look for nearby stars.
# MAST catalogs -> TESS CTL v8.01 -> Advanced Search
# Contains 9.5 million records.

# Narrow down the search as you like.

# Example filters:
# T_eff = Temperature between 2000 and 3500 K
# These are red dwarfs, which tend to have rapidly orbiting planets.
# This fits the 24-day time sampling rate of TESS.

# Distance = Between 12 and 24 parsecs
# Because we are not able to do the Kessel run in less than 12.

# Take the ID, for example: 140206488

# Double-check in EXOMAST if a planet has already been found.

# Use the ID to search in MAST and find TESS data.
# We can access this with `search_targetpixelfile()` or download the data locally.

In [ ]:
# Let's download the pixel file for a given star
# And save it to a local folder (See download helper below)

# Command to search for file - often it is not available, so that's why we download it manually
# pixelFile = search_targetpixelfile('KIC 6922244', author="TESS", cadence="long", quarter=4).download()

# Load the TESS Target Pixel File (TPF) from the local folder
tpf = TessTargetPixelFile(r"D:\VS Code\Science\Exoplanet Finder\MAST_2025-03-07T1630\TESS\tess2019357164649-s0020-0000000309661100-0165-s\tess2019357164649-s0020-0000000309661100-0165-s_tp.fits")

# Show a single snapshot of the observation at frame 42
tpf.plot(frame=42)

# Plot the light curve using the pipeline mask to select pixels
lc = tpf.to_lightcurve(aperture_mask=tpf.pipeline_mask)
lc.plot()

# Flatten the light curve to remove long-term trends
flat_lc = lc.flatten()
flat_lc.plot()

# Try to find the period of the most prominent orbiting object
# Box Least Squares (BLS) method helps to identify periodic transit signals
period = np.linspace(1, 5, 10000)
bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500)
bls.plot()

# Get the best period candidate from the periodogram
planet_x_period = bls.period_at_max_power
planet_x_t0 = bls.transit_time_at_max_power
planet_x_dur = bls.duration_at_max_power

# Phase-fold the light curve based on the discovered period at max power
ax = lc.fold(period=planet_x_period, epoch_time=planet_x_t0).scatter()
ax.set_xlim(-3, 3)